In [5]:
import os
import pandas as pd

# === 1) Load EF→ecoinvent mapping (UUID -> {TargetFlowName, TargetFlowContext}) ===
mapping_file = "ILCD-EFv3.0-ecoinventEFv3.7.xlsx"

df_map = pd.read_excel(mapping_file)
required_cols = ["SourceFlowUUID", "TargetFlowName", "TargetFlowContext"]
for col in required_cols:
    if col not in df_map.columns:
        raise ValueError(f"Missing required column in mapping file: {col}")

# Keep only rows with a non-empty TargetFlowName
df_map = df_map.dropna(subset=["TargetFlowName"])
df_map = df_map[df_map["TargetFlowName"].astype(str).str.strip() != ""]

flow_dict = {
    row["SourceFlowUUID"]: {
        "TargetFlowName": row["TargetFlowName"],
        "TargetFlowContext": row["TargetFlowContext"],
    }
    for _, row in df_map.iterrows()
}
print(f"✅ Loaded {len(flow_dict)} UUID mappings from {mapping_file}.")

# === 2) Walk through your folder structure and map per file (duplicates kept) ===
root_folder = "."  # <- change this if needed

results = {}                 # { filename: DataFrame with original + mapped columns }
unmapped_per_file = {}       # { filename: set([uuid, ...]) }
unmapped_rows = []           # list of DataFrames (unmapped rows with file column)
stats_per_file = {}          # { filename: {'rows': int, 'matched': int, 'unmapped': int, 'pct': float} }

for root, _, files in os.walk(root_folder):
    for file in files:
        if not file.endswith(('.xls', '.xlsx', '.xlsm')):
            continue

        file_path = os.path.join(root, file)
        try:
            xls = pd.ExcelFile(file_path)
            if "lciamethods_CF_GLAM" not in xls.sheet_names:
                print(f"⚠️ Sheet 'lciamethods_CF_GLAM' not found in {file}")
                continue

            # Desired columns (keep forgiving behavior)
            desired_usecols = [
                "FLOW_uuid", "FLOW_name",
                "FLOW_class0", "FLOW_class1", "FLOW_class2",
                "CF", "Unit", "CF_Uncertainty_Lower", "CF_Uncertainty_Higher",
                "LCIAMethod_location_ISO2", "LCIAMethod_type"
            ]

            df_full = pd.read_excel(file_path, sheet_name="lciamethods_CF_GLAM")
            present_cols = [c for c in desired_usecols if c in df_full.columns]
            missing_cols = [c for c in desired_usecols if c not in df_full.columns]
            if missing_cols:
                print(f"ℹ️ {file}: missing columns skipped: {missing_cols}")

            if "FLOW_uuid" not in df_full.columns:
                print(f"⚠️ No 'FLOW_uuid' column in {file}")
                continue

            df = df_full[present_cols].copy()

            # Normalize and clean UUIDs
            df["FLOW_uuid"] = df["FLOW_uuid"].astype(str).str.strip()
            df = df[df["FLOW_uuid"] != ""].dropna(subset=["FLOW_uuid"])
            df = df[df["FLOW_name"] != ""].dropna(subset=["FLOW_name"])

            # Map columns from flow_dict (preserve duplicates)
            df["TargetFlowName"] = df["FLOW_uuid"].map(lambda x: flow_dict.get(x, {}).get("TargetFlowName"))
            df["TargetFlowContext"] = df["FLOW_uuid"].map(lambda x: flow_dict.get(x, {}).get("TargetFlowContext"))

            # remove rows where mapping failed (optional)
            df = df.dropna(subset=["TargetFlowName"])
            df = df[df["TargetFlowName"] != ""].dropna(subset=["TargetFlowName"])

            # Stats per file
            total_rows = len(df)
            unmapped_rows_count = int(df["TargetFlowName"].isna().sum())
            matched_rows_count = total_rows - unmapped_rows_count
            pct_matched = (matched_rows_count / total_rows * 100.0) if total_rows else 0.0
            stats_per_file[file] = {
                "rows": total_rows,
                "matched": matched_rows_count,
                "unmapped": unmapped_rows_count,
                "pct": pct_matched,
            }

            # Identify unmapped rows (all occurrences)
            missing_mask = df["TargetFlowName"].isna()
            if missing_mask.any():
                unmapped_per_file[file] = set(df.loc[missing_mask, "FLOW_uuid"].unique())
                df_unmapped = df.loc[missing_mask].copy()
                df_unmapped["file"] = file
                unmapped_rows.append(df_unmapped)

            # Store DataFrame for Excel export
            results[file] = df

        except KeyError as ke:
            print(f"❌ Missing expected column in {file}: {ke}")
        except Exception as e:
            print(f"❌ Error reading {file}: {e}")

# === 3) Summary (with per-file % matched) ===
print("\n✅ Mapping results summary:")
total_files = len(results)
total_rows = sum(s["rows"] for s in stats_per_file.values())
total_matched = sum(s["matched"] for s in stats_per_file.values())
total_unmapped = sum(s["unmapped"] for s in stats_per_file.values())
overall_pct = (total_matched / total_rows * 100.0) if total_rows else 0.0

print(f"- Files processed: {total_files}")
print(f"- Total rows processed (duplicates kept): {total_rows}")
print(f"- Matched rows: {total_matched}")
print(f"- Unmapped rows: {total_unmapped}")
print(f"- Overall match rate: {overall_pct:.2f}%")

print("\n📊 Per-file match rate:")
for fname, s in sorted(stats_per_file.items()):
    print(f"  {fname}: {s['matched']}/{s['rows']} rows matched ({s['pct']:.2f}%)")

if unmapped_per_file:
    print("\n🔎 Unmapped UUIDs per file (unique counts):")
    for fname, s in sorted(unmapped_per_file.items()):
        print(f"  {fname}: {len(s)}")
else:
    print("\n🎉 No unmapped UUIDs found.")

# === 4) Export details of unmapped rows (CSV) ===
df_unmapped_all = None
if unmapped_rows:
    df_unmapped_all = pd.concat(unmapped_rows, ignore_index=True)
    df_unmapped_all = df_unmapped_all.sort_values(["file", "FLOW_uuid"])
    df_unmapped_all.to_csv("unmapped_flows.csv", index=False)
    print("\n💾 Wrote details of unmapped rows to 'unmapped_flows.csv'.")

# === 5) Export all per-file results to Excel ===
output_file = "flow_mapping_results.xlsx"
with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    # One sheet per processed file
    for fname, df_out in results.items():
        sheet_name = os.path.splitext(fname)[0][:31]  # Excel sheet name limit
        df_out.to_excel(writer, index=False, sheet_name=sheet_name)

    # Include a global Unmapped sheet inside the same workbook
    if df_unmapped_all is not None and not df_unmapped_all.empty:
        df_unmapped_all.to_excel(writer, index=False, sheet_name="Unmapped_Flows")

    # (Optional) also include a Summary sheet with per-file stats
    summary_rows = [
        {"file": f, "rows": s["rows"], "matched": s["matched"], "unmapped": s["unmapped"], "match_%": s["pct"]}
        for f, s in sorted(stats_per_file.items())
    ]
    if summary_rows:
        df_summary = pd.DataFrame(summary_rows)
        df_summary.to_excel(writer, index=False, sheet_name="Summary")

print(f"\n💾 Exported all mapping results to '{output_file}'")


✅ Loaded 22104 UUID mappings from ILCD-EFv3.0-ecoinventEFv3.7.xlsx.
❌ Error reading ~$flow_mapping_results.xlsx: Excel file format cannot be determined, you must specify an engine manually.
⚠️ Sheet 'lciamethods_CF_GLAM' not found in ILCD-EFv3.0-ecoinventEFv3.7.xlsx
⚠️ Sheet 'lciamethods_CF_GLAM' not found in flow_mapping_results.xlsx

✅ Mapping results summary:
- Files processed: 23
- Total rows processed (duplicates kept): 278827
- Matched rows: 278827
- Unmapped rows: 0
- Overall match rate: 100.00%

📊 Per-file match rate:
  GLAM_template_EQ_Aquatic_Microplastics.xlsx: 0/0 rows matched (0.00%)
  GLAM_template_EQ_Climate_Change_FW_TR_MA.xlsx: 99/99 rows matched (100.00%)
  GLAM_template_EQ_Ecotoxicity_FW.xlsx: 20404/20404 rows matched (100.00%)
  GLAM_template_EQ_Ecotoxicity_TR.xlsx: 2842/2842 rows matched (100.00%)
  GLAM_template_EQ_Freshwater_Eutrophication.xlsx: 6183/6183 rows matched (100.00%)
  GLAM_template_EQ_Land_Use.xlsx: 32085/32085 rows matched (100.00%)
  GLAM_template_E